In [7]:
import torch
height = width = 16

height_gap = 2 / (height - 1)
width_gap = 2 / (width - 1)
height, width = torch.meshgrid([torch.range(-1, 1, height_gap), torch.Tensor(1)])

print(torch.range(-1, 1, height_gap))
print(height)

print(width)

lr_identity = torch.stack([width, height])


print(lr_identity.shape)

tensor([-1.0000, -0.8667, -0.7333, -0.6000, -0.4667, -0.3333, -0.2000, -0.0667,
         0.0667,  0.2000,  0.3333,  0.4667,  0.6000,  0.7333,  0.8667,  1.0000])
tensor([[-1.0000],
        [-0.8667],
        [-0.7333],
        [-0.6000],
        [-0.4667],
        [-0.3333],
        [-0.2000],
        [-0.0667],
        [ 0.0667],
        [ 0.2000],
        [ 0.3333],
        [ 0.4667],
        [ 0.6000],
        [ 0.7333],
        [ 0.8667],
        [ 1.0000]])
tensor([[-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10],
        [-2.5206e+10]])
torch.Size([2, 16, 1])


/home/lrh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  
/home/lrh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  


In [8]:
a = {}
a['b'] = 2
a['c'] = 10

for k,v in a.items():
    print(k,v)

b 2
c 10


In [27]:
a = [1,2,3,4]
def func(a):
    yield a[0], a[1], a[2], a[3]
    
for i in func(a):
    print(i)

(1, 2, 3, 4)


In [10]:
a = [1,2,3,4]
print(set(a))
def func(a):
    yield a[0]
    for i in range(len(a[1:])):
        yield a[i]
    
for i in func(a):
    print(i)

{1, 2, 3, 4}
1
1
2
3


In [8]:
a = True
b = True

a and b


import random
a = random.random()
print(a < 0.5)

b = b and random.random() < 0.5

True


In [33]:
import torch
idx_data = 0
a = torch.rand([5,2,7])
print(a)
b, epoch_idx = a.max(0)
best, frame_idx = b.max(1)
print(epoch_idx)
print(best)
print(frame_idx)


best_frame_idx = frame_idx[idx_data]
print(best_frame_idx)
best_epoch_idx = epoch_idx[idx_data, frame_idx[idx_data]]
print(best_epoch_idx)

print(a[best_epoch_idx, idx_data, best_frame_idx])

tensor([[[0.1125, 0.0370, 0.6710, 0.5348, 0.2894, 0.0667, 0.8933],
         [0.3755, 0.0887, 0.2387, 0.1364, 0.8433, 0.7600, 0.1243]],

        [[0.5831, 0.8836, 0.1273, 0.1217, 0.4045, 0.5854, 0.7010],
         [0.4792, 0.2392, 0.5454, 0.1688, 0.6365, 0.1348, 0.4151]],

        [[0.3612, 0.9961, 0.6048, 0.1472, 0.2493, 0.0070, 0.1248],
         [0.5126, 0.4510, 0.1868, 0.2796, 0.9079, 0.4147, 0.4538]],

        [[0.7900, 0.3607, 0.0517, 0.5220, 0.5259, 0.4017, 0.4916],
         [0.0316, 0.8550, 0.5969, 0.3263, 0.8754, 0.4712, 0.7224]],

        [[0.6447, 0.5663, 0.3825, 0.6076, 0.1682, 0.6950, 0.5634],
         [0.5846, 0.8258, 0.8737, 0.4210, 0.7571, 0.5408, 0.4368]]])
tensor([[3, 2, 0, 4, 3, 4, 0],
        [4, 3, 4, 4, 2, 0, 3]])
tensor([0.9961, 0.9079])
tensor([1, 4])
tensor(1)
tensor(2)
tensor(0.9961)


In [44]:
a = '10'

try:
    print("frame" + str(a))
except:
    raise TypeError("a should be int type")

frame10


In [46]:
import torch
a = torch.rand([5,2,7])

print(a.shape[0])

5


In [61]:
from tqdm import tqdm
import numpy as np
from time import sleep
for idx, i in enumerate(tqdm(np.arange(100))):
    print(idx)
    sleep(0.01)

 20%|██        | 20/100 [00:00<00:00, 98.14it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


 30%|███       | 30/100 [00:00<00:00, 98.29it/s]

20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


 50%|█████     | 50/100 [00:00<00:00, 98.17it/s]

...

 70%|███████   | 70/100 [00:00<00:00, 98.03it/s]

60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


 90%|█████████ | 90/100 [00:00<00:00, 98.01it/s]

80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


100%|██████████| 100/100 [00:01<00:00, 98.00it/s]
